### geocoding and plot

In [25]:
import geopandas
import geopandas.tools as gts


ModuleNotFoundError: No module named 'geopandas'

In [4]:
prefs = [
    '北海道', '青森県', '岩手県', '宮城県', '秋田県', '山形県',
    '福島県', '茨城県', '栃木県', '群馬県', '埼玉県', '千葉県',
    '東京都', '神奈川県', '新潟県', '富山県', '石川県', '福井県',
    '山梨県', '長野県', '岐阜県', '静岡県', '愛知県', '三重県',
    '滋賀県', '京都府', '大阪府', '兵庫県', '奈良県', '和歌山県',
    '鳥取県', '島根県', '岡山県', '広島県', '山口県', '徳島県', 
    '香川県', '愛媛県', '高知県', '福岡県', '佐賀県', '長崎県', 
    '熊本県', '大分県', '宮崎県', '鹿児島県', '沖縄県'
]


In [5]:
codes = gts.geocode(prefs, provider='nominatim', user_agent='test')
# save to GeoJSON
#outpath = './prefs.geojson'
#codes.to_file(outpath, driver='GeoJSON', encoding='utf-8')


NameError: name 'gts' is not defined

In [6]:
codes.plot()

NameError: name 'codes' is not defined

In [7]:
pref = codes["address"][0]
# lon
lon = codes["geometry"][0].x
# lat
lat = codes["geometry"][0].y

f'{pref}の緯度経度: ({lon},{lat})'

NameError: name 'codes' is not defined

In [8]:
codes['lon'] = codes['geometry'].map(lambda x: x.x)
codes['lat'] = codes['geometry'].map(lambda x: x.y)


NameError: name 'codes' is not defined

In [9]:
import random
codes['rand'] = codes['geometry'].map(lambda x: random.randint(1, 10))

NameError: name 'codes' is not defined

In [10]:
codes.head()

NameError: name 'codes' is not defined

In [11]:
import json
geojson = json.loads(codes.to_json())

for feat in geojson['features']:
    prop = feat['properties']
    coor = feat["geometry"]["coordinates"]
    print(f"{prop['address']}: {coor}")\
    
    break

NameError: name 'codes' is not defined

In [12]:
import pydeck as pdk
from pydeck.types import String

# layer option https://deck.gl/docs/api-reference/aggregation-layers/heatmap-layer
layer = pdk.Layer(
    'HeatmapLayer', # 'HexagonLayer', # 'ScatterplotLayer', #'HeatmapLayer', 
    codes,
    get_position=['lon', 'lat'],
    #auto_highlight=True,
    get_radius=100,
    #get_fill_color='[180, 0, 200, 140]',
    pickable=True,
    opacity=0.7,
    radiusPixels=20,
    aggregation=String('MEAN'),
    get_weight="rand"
)

# 座標参考: https://www.gsi.go.jp/KOKUJYOHO/center.htm
view_state = pdk.ViewState(
    longitude=137.5936,
    latitude=36.047,
    zoom=5,
    min_zoom=1,
    max_zoom=15,
    pitch=50,
    bearing=-0
)

# Combined all of it and render a viewport
r = pdk.Deck(
    #map_style='mapbox://styles/mapbox/light-v9',
    layers=[layer], initial_view_state=view_state,
    #map_provider=None
)

r.to_html('example.html')

ModuleNotFoundError: No module named 'pydeck'

In [13]:
# https://deckgl.readthedocs.io/en/latest/layer.html#the-type-positional-argument
import pydeck as pdk

UK_ACCIDENTS_DATA = 'https://raw.githubusercontent.com/visgl/deck.gl-data/master/examples/3d-heatmap/heatmap-data.csv'

layer = pdk.Layer(
    'HexagonLayer',  # `type` positional argument is here
    UK_ACCIDENTS_DATA,
    get_position=['lng', 'lat'],
    auto_highlight=True,
    elevation_scale=50,
    pickable=True,
    elevation_range=[0, 3000],
    extruded=True,
    coverage=1)

# Set the viewport location
view_state = pdk.ViewState(
    longitude=-1.415,
    latitude=52.2323,
    zoom=6,
    min_zoom=5,
    max_zoom=15,
    pitch=40.5,
    bearing=-27.36)

# Combined all of it and render a viewport
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html('hexagon-example.html')

ModuleNotFoundError: No module named 'pydeck'

### use geojson

In [14]:
# https://note.com/yearman/n/n69fa3f2d583d
japan = geopandas.read_file('https://raw.githubusercontent.com/dataofjapan/land/master/japan.topojson')
japan.plot()



NameError: name 'geopandas' is not defined

In [15]:
japan[japan['nam_ja'] == '北海道'].plot()


NameError: name 'japan' is not defined

In [16]:
japan.head()

NameError: name 'japan' is not defined

In [17]:
japan['geometry'][1]

NameError: name 'japan' is not defined

In [18]:
japan['geometry'][2]

NameError: name 'japan' is not defined

In [19]:
import random
japan['rand'] = japan['geometry'].map(lambda x: random.randint(0, 10))

NameError: name 'japan' is not defined

In [20]:
import pydeck as pdk

# 座標参考: https://www.gsi.go.jp/KOKUJYOHO/center.htm
view_state = pdk.ViewState(
    longitude=137.5936,
    latitude=36.047,
    zoom=4,
    min_zoom=1,
    max_zoom=15,
    pitch=50,
    bearing=-0
)

geojson = pdk.Layer(
    "GeoJsonLayer",
    japan,
    opacity=0.8,
    stroked=False,
    filled=True,
    extruded=True,
    wireframe=True,
    get_fill_color="[255, rand * 255, 255]",
    get_line_color=[255, 255, 255],
)

r = pdk.Deck(layers=[geojson], initial_view_state=view_state)

r.to_html("geojson_layer.html")

ModuleNotFoundError: No module named 'pydeck'